In [1]:
import joblib

cv, x_train, x_test, y_train, y_test = joblib.load('nsmc2.pkl')

### 모형

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.Sequential()

In [6]:
model.add(  
    tf.keras.layers.Dense(
        16, 
        input_shape=(3048,),
        activation='relu',
        name='hidden'
    )
)

In [7]:
model.add(
    tf.keras.layers.Dense(
        1,  
        activation='sigmoid',
        name='output'
    )
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 16)                48784     
_________________________________________________________________
output (Dense)               (None, 1)                 17        
Total params: 48,801
Trainable params: 48,801
Non-trainable params: 0
_________________________________________________________________


### 학습률

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

### 학습

In [11]:
model.fit(x_train.toarray(),y_train.values, epochs=100, validation_split=0.1,
         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')]) 

Train on 7382 samples, validate on 821 samples
Epoch 1/100
7382/7382 [==============================] - 0s 62us/sample - loss: 0.3934 - accuracy: 0.8763 - val_loss: 0.3668 - val_accuracy: 0.8782
Epoch 2/100
7382/7382 [==============================] - 0s 38us/sample - loss: 0.2516 - accuracy: 0.8950 - val_loss: 0.3627 - val_accuracy: 0.8794
Epoch 3/100
7382/7382 [==============================] - 0s 35us/sample - loss: 0.1942 - accuracy: 0.9218 - val_loss: 0.3826 - val_accuracy: 0.8782


### 모형평가

In [12]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.29708051237244537, 0.887372]

# hyperopt

### 탐색공간 정의하기

In [13]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

In [16]:
space = hp.choice('model',[
    {
        'layer': 'single',
        'lr': hp.loguniform('lr1',-7,2)  
    },
    {
        'layer': 'multi',
        'lr': hp.loguniform('lr2',-7,2),
        'units': hp.quniform('units',16,128,16), 
        'activation': hp.choice('activation',['relu','tanh'])
    }
])

In [18]:
import joblib

cv, x_train, x_test, y_train, y_test = joblib.load('nsmc2.pkl')

### 모형

In [19]:
import tensorflow as tf
from hyperopt import STATUS_OK
import uuid

In [21]:
def objective(h):
    model = tf.keras.models.Sequential()
    
    if h['layer'] == 'multi':
        model.add(tf.keras.layers.Dense(h['units'], activation=h['activation']))  # input_shape 자동지정댐.
    
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=h['lr']),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(
        x_train.toarray(),
        y_train.values,
        epochs=100,
        verbose=0, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')]
    )
    
    loss, acc = model.evaluate(x_test.toarray(), y_test.values, verbose=0)
    
    filename = str(uuid.uuid4())
    model.save(filename)
    
    return{
        'loss': -acc,  
        'status': STATUS_OK,
        'filename': filename
    }

In [22]:
h = sample(space)

### 탐색

In [23]:
import warnings

In [24]:
warnings.simplefilter('ignore')  
tf.get_logger().setLevel('ERROR')  

In [25]:
objective(h)

{'loss': -0.8537299,
 'status': 'ok',
 'filename': '9b0e9725-ebcd-43a8-9d71-a89728372ef9'}

In [26]:
from hyperopt import fmin, rand, tpe, Trials

In [27]:
trials = Trials()

In [28]:
fmin(
    objective,
    space=space,
    algo=rand.suggest,  # rand는 random하게 탐색하는 알고리즘 사용, tpe는 이전의 경험을 토대로 발견.
    max_evals=3,
    trials=trials
)

100%|█████████████████████████████████████████████████| 3/3 [01:07<00:00, 22.58s/trial, best loss: -0.8620185256004333]


{'activation': 1, 'lr2': 0.025027610402074456, 'model': 1, 'units': 48.0}

In [30]:
best_model = tf.keras.models.load_model(trials.best_trial['result']['filename'])

In [31]:
best_model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.7913424538322218, 0.8620185]